In [1]:
# The script gets eQTLs of a gene (HNF4a, for now), saves them sorted
# and merged (in a two different files); Intersects it with ASBs for
# all the TFs (which are stored in adastra_snps/all_tfs.bed)

In [2]:
import json
import requests
import subprocess
import bedwriter

In [3]:
request = requests.get(f'https://adastra.autosome.ru/api/v4/search/snps/eqtl_gene_id/ENSG00000101076')
eqtls = json.loads(request.text)['results']

In [4]:
window_halfwidth = 2000
eqtl_tuples = []
for eqtl in eqtls:
    eqtl_tuples.append((eqtl['chromosome'],
                        eqtl['position'] - 1 - window_halfwidth,
                        eqtl['position'] + window_halfwidth))
bedwriter.write_tuples_in_bed(eqtl_tuples, "eqtls/HNF4A-eqtl.bed")
print(*eqtl_tuples, sep='\n', end='\n\n')

# sort and merge intersecting intervals
sorted_bed = subprocess.run(["bedtools", "sort", "-i", "eqtls/HNF4A-eqtl.bed"],
                            capture_output=True).stdout
with open( "eqtls/HNF4A-eqtl.bed", 'wb') as f:
    f.write(sorted_bed)
    
merged_bed = subprocess.run(["bedtools", "merge", "-i", "eqtls/HNF4A-eqtl.bed"],
                            capture_output=True).stdout
with open( "eqtls/HNF4A-eqtl-merged.bed", 'wb') as f:
    f.write(merged_bed)
    
print(merged_bed.decode())

('chr20', 44409210, 44413211)
('chr20', 44409496, 44413497)
('chr20', 44409647, 44413648)
('chr20', 44409552, 44413553)

chr20	44409210	44413648



In [5]:
intersection = subprocess.run(["bedtools", "intersect", "-a", "adastra_snps/all_tfs.bed",
                               "-b", "eqtls/HNF4A-eqtl-merged.bed", "-wa"], 
                              capture_output=True).stdout
print(intersection.decode())

chr20	44410399	44410400	RBM22_HUMAN
chr20	44411210	44411211	RBM22_HUMAN
chr20	44411552	44411553	RARA_HUMAN



In [6]:
intersection = subprocess.run(["bedtools", "intersect", "-a", "adastra_snps/all_tfs.bed",
                               "-b", "eqtls/HNF4A-eqtl-merged.bed", "-wb"], 
                              capture_output=True).stdout
print(intersection.decode())

chr20	44410399	44410400	RBM22_HUMAN	chr20	44409210	44413648
chr20	44411210	44411211	RBM22_HUMAN	chr20	44409210	44413648
chr20	44411552	44411553	RARA_HUMAN	chr20	44409210	44413648



In [7]:
#or now possible:
import asbs_near_eqtls
asbs_near_eqtls.find_asbs_near_eqtls('ENSG00000101076', 'HNF4A')

ASBs near eQTLs with target HNF4A:
chr20	44410399	44410400	RBM22_HUMAN
chr20	44411210	44411211	RBM22_HUMAN
chr20	44411552	44411553	RARA_HUMAN

